In [1]:
%load_ext autoreload
%autoreload 2

## Imports


In [2]:
from pathlib import Path

import torch

In [21]:
DATA_DIR = Path("../../../elk-reporters/gpt2/imdb/gracious-jennings/")
DATA_DIR

PosixPath('../../../elk-reporters/gpt2/imdb/gracious-jennings')

In [22]:
!ls {DATA_DIR}

cfg.yaml  fingerprints.yaml  lr_eval.csv  reporters	  train_lm_eval.csv
eval.csv  lm_eval.csv	     lr_models	  train_eval.csv


## Load VINC model

In [23]:
files = [f for f in (DATA_DIR / "reporters").iterdir() if f.suffix == ".pt"]
files = sorted(files, key=lambda x: int(x.stem.split("_")[-1]))
len(files), files[:5]

(1,
 [PosixPath('../../../elk-reporters/gpt2/imdb/gracious-jennings/reporters/layer_12.pt')])

In [24]:
for f in files[:2]:
    print(f.name)
    tns = torch.load(f)
    print(tns["num_classes"])
    print(tns["in_features"])

    for k, v in tns.items():
        if isinstance(v, torch.Tensor):
            print(k, v.shape)
    
    print("-------------\n")

layer_12.pt
2
768
bias torch.Size([1])
scale torch.Size([1])
weight torch.Size([1, 768])
norm.mean_x torch.Size([768])
norm.mean_y torch.Size([2])
norm.xcov_M2 torch.Size([768, 2])
norm.n torch.Size([])
-------------



## Try and load a VINC probe

In [25]:
ELK_PATH = Path("../../../elk/")
ELK_PATH.resolve()

PosixPath('/rds/user/am3052/hpc-work/elk')

In [26]:
import sys
sys.path

['/rds/user/am3052/hpc-work/elk/elk/training',
 '/rds/user/am3052/hpc-work/elk',
 '/rds/user/am3052/hpc-work/elk-rlhf/elk/elk/training',
 '/rds/user/am3052/hpc-work/elk-rlhf/elk',
 '/rds/user/am3052/hpc-work/elk-rlhf/src/prototyping',
 '/home/am3052/.conda/envs/elk/lib/python310.zip',
 '/home/am3052/.conda/envs/elk/lib/python3.10',
 '/home/am3052/.conda/envs/elk/lib/python3.10/lib-dynload',
 '',
 '/home/am3052/.conda/envs/elk/lib/python3.10/site-packages',
 '__editable__.eleuther_elk-0.1.1.finder.__path_hook__']

In [27]:
modules = [ELK_PATH, ELK_PATH / "elk" / "training"]

for module in modules:
    if not str(module) in sys.path:
        sys.path.insert(0, str(module.resolve()))

sys.path

['/rds/user/am3052/hpc-work/elk/elk/training',
 '/rds/user/am3052/hpc-work/elk',
 '/rds/user/am3052/hpc-work/elk/elk/training',
 '/rds/user/am3052/hpc-work/elk',
 '/rds/user/am3052/hpc-work/elk-rlhf/elk/elk/training',
 '/rds/user/am3052/hpc-work/elk-rlhf/elk',
 '/rds/user/am3052/hpc-work/elk-rlhf/src/prototyping',
 '/home/am3052/.conda/envs/elk/lib/python310.zip',
 '/home/am3052/.conda/envs/elk/lib/python3.10',
 '/home/am3052/.conda/envs/elk/lib/python3.10/lib-dynload',
 '',
 '/home/am3052/.conda/envs/elk/lib/python3.10/site-packages',
 '__editable__.eleuther_elk-0.1.1.finder.__path_hook__']

In [28]:
from reporter import Reporter

In [29]:
reporter_path = files[-1]
reporter_path.resolve()

PosixPath('/rds/user/am3052/hpc-work/elk-reporters/gpt2/imdb/gracious-jennings/reporters/layer_12.pt')

In [30]:
reporter = Reporter.load(reporter_path)
reporter

EigenReporter(
  (norm): ConceptEraser()
)

In [31]:
reporter.weight.shape

torch.Size([1, 768])

In [32]:
reporter.scale, reporter.bias

(Parameter containing:
 tensor([5.9876], requires_grad=True),
 Parameter containing:
 tensor([-0.0002], requires_grad=True))